In [9]:
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

In [14]:
kld = pd.read_csv('../../data/processed/kolding_features.csv', index_col='Datetime', parse_dates=['Datetime'])
train = kld.copy()

In [15]:
T = 6
HORIZON = 1
valid_date = dt.datetime(2021,1,1)
test_date = dt.datetime(2021,2,1)

In [16]:
yscaler = MinMaxScaler()
yscaler.fit(train[['Value']])

MinMaxScaler()

In [18]:
Xscaler = MinMaxScaler()
train[kld.columns.tolist()] = Xscaler.fit_transform(train)

In [ ]:




y_train = train_shifted[y_col].values
X_train = train_shifted[X_cols].values

X_train = X_train.reshape(X_train.shape[0], T, 1)

print(y_train.shape)
print(X_train.shape)
val = kld.copy()[(kld.index >= valid_date) & (kld.index < end_date)]
val['Value'] = scaler.fit_transform(val)
val_shifted = val.copy()
val_shifted['y_t+1'] = val_shifted['Value'].shift(-1, freq='H')
for t in range(1, T+1):
    val_shifted[str(T-t)] = val_shifted['Value'].shift(T-t, freq='H')
y_col = 'y_t+1'
X_cols = ['load_t-5',
         'load_t-4',
         'load_t-3',
         'load_t-2',
         'load_t-1',
         'load_t']
val_shifted.columns = ['load_original'] + [y_col] + X_cols

val_shifted = val_shifted.dropna(how='any')
val_shifted.head()
y_val = val_shifted[y_col].values
X_val = val_shifted[X_cols].values

X_val = X_val.reshape(X_val.shape[0], T, 1)

print(y_val.shape)
print(X_val.shape)

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

LATENT_DIM = 5
BATCH_SIZE = 32
EPOCHS = 25

model = Sequential()
model.add(GRU(LATENT_DIM, input_shape=(T, 1)))
model.add(Dense(HORIZON))
model.compile(optimizer='RMSprop', loss='mse')
model.summary()

GRU_ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

hist = model.fit(X_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(X_val, y_val),
                callbacks=[GRU_ES],
                verbose=1)

preds = model.predict(X_val)

eval_data = pd.DataFrame(preds, columns=['t+'+str(t) for t in range(1, HORIZON+1)])
eval_data['Datetime'] = val_shifted.index
eval_data = pd.melt(eval_data, id_vars='Datetime', value_name='Prediction', var_name='H')
eval_data['Actual'] = np.transpose(y_val).ravel()
eval_data[['Prediction','Actual']] = scaler.inverse_transform(eval_data[['Prediction','Actual']])
eval_data.head()

def mape(pred, true):
    return ((pred - true).abs() / true).mean()

1-mape(eval_data['Prediction'], eval_data['Actual'])